In [2]:
def rule(name: str, match: str, build: str, where: str):
    return rule(name, match, build) + f"where {where}"

def rule(name: str, match: str, build: str):
    return f"{name}: {match} -> {build}"

In [51]:
MODULE_NAME = "benchnullary"
RULE_NAME = "R"
SORT_NAME = "S"
CONS_NAME = "Constr{}"
BUILD = "{}"

PROGRAM_SIZE_LOWER = 100
PROGRAM_SIZE_UPPER = 1000
PROGRAM_SIZE_STEP = 100

PROGRAM_OUTPUT_DIR = "generated-benchmarks"

In [52]:
import os

def PROGRAM(input_size: int, num_constructors: int):
    constructor_ids = range(1, num_constructors + 1)
    module_name = MODULE_NAME + str(input_size)
    return {
        "module_name": module_name, 
        "contents": f"""
module {module_name}

signature
  sorts {SORT_NAME}
  constructors
{os.linesep.join([f"    {CONS_NAME.format(i)} : {SORT_NAME}" for i in constructor_ids])}

strategies

  main = <{RULE_NAME}> {CONS_NAME.format(input_size)}()


rules

{os.linesep.join([f"  {rule(RULE_NAME, CONS_NAME.format(i) + '()', str(i))}" for i in constructor_ids])}

"""}

print(PROGRAM(2, 4)["contents"])


module benchnullary2

signature
  sorts S
  constructors
    Constr1 : S
    Constr2 : S
    Constr3 : S
    Constr4 : S

strategies

  main = <R> Constr2()


rules

  R: Constr1() -> 1
  R: Constr2() -> 2
  R: Constr3() -> 3
  R: Constr4() -> 4




In [54]:
from os import path

for input_size in range(PROGRAM_SIZE_LOWER, PROGRAM_SIZE_UPPER + 1, PROGRAM_SIZE_STEP):
    prog = PROGRAM(input_size, PROGRAM_SIZE_UPPER)
    
    filename = path.join(PROGRAM_OUTPUT_DIR, f"{prog['module_name']}.str2")
    contents = prog["contents"]
    
    os.makedirs(path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as prog_file:
        prog_file.write(contents)